In [1]:
#This code imports packages that will be used later on, allowing us to use SeQUeNCe's modules and capabilities

#Import the Node and BSMNode classes, which we will use as the basis for our player's circuits and referees circuit, respectively
from sequence.topology.node import Node, BSMNode

#Import the Memory class, which we will use to hold our qubits
from sequence.components.memory import Memory

#Import the EntanglementGenerationA class, which we will use to entangle our player's qubits
from sequence.entanglement_management.generation import EntanglementGenerationA

#Import the Timeline class, which will allow our simulation to run
from sequence.kernel.timeline import Timeline

#Import the QuantumChannel and ClassicalChannel classes, which allow for communication between Alice and Bob and the referee
from sequence.components.optical_channel import QuantumChannel, ClassicalChannel

#Import the EntanglementProtocol class, which manages the processes of creating entanglement
from sequence.entanglement_management.entanglement_protocol import EntanglementProtocol

#Import the Message class, which enables communication on classical channels
from sequence.message import Message

#Import the Circuit class, which we will use to build custom circuits
from sequence.components.circuit import Circuit, x_gate, y_gate, z_gate, s_gate, t_gate, validator

#Import relevant components from Qutip, a quantum system simulator that SeQUeNCe incorporates
from qutip.qip.circuit import QubitCircuit
from qutip.qip.operations import gate_sequence_product
from qutip import Qobj

In [2]:
#This code defines a custom class for our entangled nodes, as well a resource manager for that class
#The resource manager is needed because SeQUeNCe's implementation of entanglement management requires a resource_manager with an update function

#NEED TO ADD COMMENTS TO THIS CELL
class Manager:
    def __init__(self, node, mem_name):
        self.node = node
        self.mem_name = mem_name
        
    def update(self, prot, mem, st):
        if st == 'RAW':
            mem.reset()
    
    def add_entanglement_protocol(self, middle: str, other: str):
        self.node.protocols = [EntanglementGenerationA(self.node, '%s.eg' % self.node.name, 
                                                       middle, other, self.node.components[self.mem_name])]

class PlayerNode(Node):
    def __init__(self, name: str, tl: Timeline):
        super().__init__(name, tl)
        
        mem_name = '%s.mem' % name
        mem = Memory(mem_name, tl, fidelity = 1, frequency = 0,
                    efficiency = 1, coherence_time = 0, wavelength = 500)
        mem.owner = self
        mem.add_receiver(self)
        self.add_component(mem)
        self.resource_manager = Manager(self, mem_name)
        
        self.circ = CustomCircuit(0)
        
    def init(self):
        mem = self.get_components_by_type('Memory')[0]
        mem.reset()
        
    def receive_msg(self, src: str, msg: 'Message'):
        self.protocols[0].received_message(src, msg)
    
    def get(self, photon, **kwargs):
        self.send_qubit(kwargs['dst'], photon)
        
def gate_0():
    angle_0 = np.pi/8
    mat = np.array([[np.cos(angle_0), np.sin(angle_0)], 
                    [np.sin(angle_0), -np.cos(angle_0)]])
    return Qobj(mat, dims = [[2], [2]])

def gate_1():
    angle_1 = -np.pi/8
    mat = np.array([[np.cos(angle_1), np.sin(angle_1)], 
                    [np.sin(angle_1), -np.cos(angle_1)]])
    return Qobj(mat, dims = [[2], [2]])

class CustomCircuit(Circuit):
    def __init__(self, size: int):
        super().__init(size)
        
    def get_unitary_matrix(self) -> np.ndarray:
        if self._cache is None:
            if len(self.gates) == 0:
                self._cache = np.identity(2 ** self.size)
                return self._cache

            qc = QubitCircuit(self.size)
            qc.user_gates = {"X": x_gate,
                             "Y": y_gate,
                             "Z": z_gate,
                             "S": s_gate,
                             "T": t_gate,
                             "0": gate_0,
                             "1": gate_1}
            for gate in self.gates:
                name, indices, arg = gate
                if name == 'h':
                    qc.add_gate('SNOT', indices[0])
                elif name == 'x':
                    qc.add_gate('X', indices[0])
                elif name == 'y':
                    qc.add_gate('Y', indices[0])
                elif name == 'z':
                    qc.add_gate('Z', indices[0])
                elif name == 'cx':
                    qc.add_gate('CNOT', controls=indices[0], targets=indices[1])
                elif name == 'ccx':
                    qc.add_gate('TOFFOLI', controls=indices[:2], targets=indices[2])
                elif name == 'swap':
                    qc.add_gate('SWAP', indices)
                elif name == 't':
                    qc.add_gate('T', indices[0])
                elif name == 's':
                    qc.add_gate('S', indices[0])
                elif name == 'phase':
                    qc.add_gate('PHASEGATE', indices[0], arg_value=arg)
                elif name == '0_gate':
                    qc.add_gate('0', indices[0])
                elif name == '1_gate':
                    qc.add_gate('1', indices[0])
                else:
                    raise NotImplementedError
            self._cache = gate_sequence_product(qc.propagators()).full()

        return self._cache

In [6]:
#This code creates the infrastructure for our simulation

#Create the timeline for the simulation
tl = Timeline()

#Create nodes for Alice, Bob, and the ref
alice = PlayerNode('alice', tl)
bob = PlayerNode('bob', tl)

#The referee's channel uses SeQUeNCe's built-in BSMNode
ref = BSMNode('ref', tl, ['alice', 'bob'])
nodes = [alice, bob, ref]

#Set the efficiency of the BSM to 1, which means no errors
bsm = ref.get_components_by_type('SingleAtomBSM')[0]
bsm.update_detectors_params('efficiency', 1)

#Create quantum channels between Alice and Bob and the ref
qcA = QuantumChannel('qcA', tl, attenuation = 0, distance = 1000)
qcB = QuantumChannel('qcB', tl, attenuation = 0, distance = 1000)
qcA.set_ends(alice, ref.name)
qcB.set_ends(bob, ref.name)

#Create classical channels between all three participants
#Classical channels are one way only, so we have to make two channels for each connection
for i in range (len(nodes)):
    for j in range(len(nodes)):
        if (i != j):
            cc = ClassicalChannel('cc_%s_%s'%(nodes[i].name, nodes[j].name), tl, 1000, 1e8)
            cc.set_ends(nodes[i], nodes[j].name)

In [7]:
#This code creates and pairs the protocols for generating entanglement

#NEED TO ADD COMMENTS TO THIS CELL
def pair_protocol(node1: Node, node2: Node):
    p1 = node1.protocols[0]
    p2 = node2.protocols[0]
    n1_mem_name = node1.get_components_by_type('Memory')[0].name
    n2_mem_name = node2.get_components_by_type('Memory')[0].name
    p1.set_others(p2.name, node2.name, [n2_mem_name])
    p2.set_others(p1.name, node1.name, [n1_mem_name])
    
alice.resource_manager.add_entanglement_protocol('ref', 'bob')
bob.resource_manager.add_entanglement_protocol('ref', 'alice')
pair_protocol(alice, bob)

mem = alice.get_components_by_type('Memory')[0]

In [8]:
#This code runs the simulation
tl.init()
alice.protocols[0].start()
bob.protocols[0].start()
tl.run()

print(mem.entangled_memory)

{'node_id': 'bob', 'memo_id': 'bob.mem'}


In [ ]:
#FIGURING OUT GATES AND THEN IMPLEMENTING A REF NODE FOR JUST CLASSICAL COMMUNICATION -- HAVE TO REWORK SOME OF THE ABOVE
from enum import Enum
from sequence import message
import random
import numpy as np

class MsgType(Enum):
    ZERO = 0
    ONE = 1
    READY = 2
    
class Player(Enum):
    ALICE = 0
    BOB = 1
    
class RefereeProtocol(Protocol):
    def __init__(self, node: Node, name: str, rec_name: str, rec_node: str):
        super().__init__(node, name)
        node.protocols.append(self)
        self.rec_name = rec_name
        self.rec_node = rec_node
        self.msgs_rec = 0
        
    def init(self):
        pass
    
    def sendBit(self):
        bit = random.randint(0, 1)
        msg = Message(MsgType(bit), self.rec_name)
        self.node.send_message(self.rec_name, msg)
    
    def received_message(self, src: str, msg: Message):
        print("node {} received {} message from {}").format(self.node.name, msg, src)
        self.msgs_rec += 1
        if (msgs.rec == 2):
            
        
class PlayerProtocol(Protocol):
    def __init__(self, node: Node, name: str, rec_name: str, rec_node: str):
        super().__init__(node, name)
        node.protocols.append(self)
        self.rec_name = rec_name
        self.rec_node = rec_node
    
    def init(self):
        pass
    
    def received_message(self, src: str, msg: Message):
        print("node {} received {} message from {}").format(self.node.name, msg, src)
        self.node.applyGate(self, msg)
        msg = Message(MsgType(2), src)
        self.node.send_message(src, msg)
        
#DEFINE PROTOCOLS FOR ALICE AND BOB'S GATES
class AliceProtocol(Protocol):
    def __init__(self, node: Node, name: str):
        super().__init__(node, name)
        node.protocols.append(self)
        
    def init(self):
        pass
        
    def applyGate(self, in_bit: Message):
        #Apply Hadamard gate and measure if input = 1, otherwise just measure
        if (in_bit == MsgType.ONE):
            self.node.circ.gates.append(['h', [0], None])
        
class BobProtocol(Protocol):    
    def __init__(self, node: Node, name: str):
        super().__init__(node, name)
        node.protocols.append(self)
        
    def init(self):
        pass
            
    def applyGate(self, in_bit: Message):
        #Apply the unitary gate that corresponds to the input
        
        #MAKE SURE THIS WORKS -- MIGHT BE DOING THIS WRONG
        if (in_bit == MsgType.ONE):
            self.node.circ.gates.append(['1_gate', [0], None])
        else:
            self.node.circ.gates.append(['0_gate', [0], None])
        

WANT TO SHOW HOW CHANGING FIDELITY, USING NETWORK MANAGER, ETC. AFFECTS THEIR SUCCESS

STEPS:
- Timeline is initialized
- Start game function is called to add events to the timeline (not sure if I should use protocols or processes) 
    - custom defining process may work better
    - however, rule conditions of protocols may make them a natural fit
    - ok i get it now: processes are used to start protocols (part 2 of example 2). each step along the way is a protocol.
- Timeline is run

GAME STEPS:
- Alice and Bob entangle 
- Ref sends bit to both players 
- Both players receive their bit, apply their respective gates, then measure their gates themselves (with get() method)
- Both players return a bit to the ref
- Ref declares a win or loss
- Game is reset

DOING:

Need to update how I'm measuring circuits. I think I have to do all of this custom, after entaglement. Create a CustomCircuit class so I can override one get_unitary_matrix() function and add the custom gate I need. Then, similar to a detector, go through how I run the circuit (should be simpleish, just like run_circuit method or something). Get rid of the parts where I use a detector; that won't help me.

The files in the opensource i am looking at that are helpful are: detector, circuit, quantum_manager

Use measure_multiple to measure -- not sure how the version in Photon and in QuantumState interact.

CIRCUIT STEPS:
- Assign the quantum state of the photons in memory of each player node to two variables, and put those two states in a list
- Get the basis vectors for each qubit with the get_unitary_matrix method and put those in a list
- Call measure_multiple on those, w rng